In [1]:
import numpy as np

class bond(object):
    def __init__(self, libor, coupon, t_series):
        self.libor=libor
        self.coupon=coupon
        self.t_series = t_series
        self.ntimes=len(self.t_series)
        self.pvAvg=0.0

    def pv(self):
        deltaT= self.t_series[2:]-self.t_series[-1:]
        self.cashFlows= self.coupon*deltaT
        principal = np.zeros(deltaT)
        principal[-1:]=1.0
        self.cashFlows += principal
        pv = self.cashFlows*self.libor[2:]
        self.pvAvg = np.average(pv,axis=2)
        return

In [2]:
__author__ = 'marcopereira'
from pandas import DataFrame, Series
import numpy as np
import pandas as pd


class MC_Vasicek_Sim(object):
    def __init__(self, date_list,r0,sigmaR,muR,alphaR, simNumber,t_step):
    #SDE parameters - Vasicek SDE
        self.sigmaR = sigmaR
        self.muR = muR
        self.alphaR = alphaR
        self.simNumber = simNumber
        self.t_step = t_step
        self.r0 = r0
    #internal representation of times series - integer multiples of t_step
        self.t_series = pd.DataFrame(date_list)
    #creation of a fine grid for Monte Carlo integration
        #Create fine date grid for SDE integration
        minDay = min(date_list)
        maxDay = max(date_list)
        self.datelistlong = pd.date_range(minDay, maxDay).tolist()
        self.ntimes = len(self.datelistlong)
        return

    def getLibor(self):
        rd = np.random.random((self.ntimes,self.simNumber))   # array of numbers for the number of samples
        r = np.ones(np.shape(rd))
        r[0,:] = self.r0*r[0,:]
        nrows = np.shape(rd)[0]
        sigmaDT = self.sigmaR* np.sqrt(self.t_step)
    #calculate r(t)
        r[0,:] = self.r0*r[0,:]
        for i in np.arange(1,nrows):
            r[i,:] = r[i-1,:]+ self.alphaR*(self.muR-r[i-1,:]) + sigmaDT*rd[i,:]
    #calculate integral(r(s)ds)
        integralR = r.cumprod(axis=0)*self.t_step
    #calculate Libor
        self.libor = np.exp(-integralR)
        return

    def return_indices1_of_a(self, a, b):
        b_set = set(b)
        return [i for i, v in enumerate(a) if v in b_set]

    def return_indices2_of_a(self, a, b):
        index=[]
        for item in a:
            index.append(np.bisect.bisect(b,item))
        return np.unique(index).tolist()

    def getSmallLibor(self):
        #calculate indexes
        ind = self.return1_indices(self.date_list, self.datelistlong)
        return self.libor[ind,:]

In [3]:
__author__ = 'marcopereira'
import numpy as np
import datetime


#CashFlow Dates
t_series = np.round(np.arange(0,0.25001,0.25)*365)
base = datetime.datetime.today()
datelist = [base + datetime.timedelta(days=x) for x in t_series]

#SDE parameter
t_step = 1.0/365
r0 = 0.08
sigmaR = 0.05
muR = 0.05
alphaR=3.0
simNumber=10

#Bond parameters
coupon = 0.08

In [4]:
#Monte Carlo trajectories creation
myVasicek = MC_Vasicek_Sim(datelist, r0,sigmaR, muR, alphaR, simNumber,t_step).getLibor()

In [ ]:
libor = myVasicek.getSmallLibor()

In [5]:
myVasicek